In [1]:
from __future__ import division
from datetime import datetime

In [2]:
dateparser=lambda x:datetime.strptime(x,
                                      "%Y-%m-%d") if "-" in x else datetime.strptime(x,
                                                                                     "%m/%d/%Y")

In [3]:
summov=lambda x,win:[sum(x[i:win+i])for i in range(len(x[win-1:]))]

In [4]:
from math import sqrt,log,e
def gammafit (data):
    n=len(data)
    if n==0:
        return 0,0
    while 0 in data:
        data.remove(0)
    nact=len(data)
    if  nact>0:
        mn=sum(data)/nact
    if nact==1 or sum(data)==0 or sum(data)/nact==data[0]:
        alpha,gamm,mn=0.0,1.0,mn
        return gamm, mn
    sumlog=sum([log(i)for i in data])
    A=log(mn)-sumlog/nact
##    print(data)
    k=(1.0+sqrt(1.0+4.0*A/3.0))/(4.0*A)
    theta=mn/k
    return k,theta


In [5]:
def lower_incomplete_gamma(a,x,d=0,iterations=100):
    if d == iterations:
        if ((d % 2) == 1):
            return 1.0 # end iterations
        else:
            m = d/2
            return x + (m-a)
    if d == 0:
        result = ((x**a) * (e**(-x)))/lower_incomplete_gamma(a,x,d=d+1)
        return result
    elif ((d % 2) == 1):
        m = d - 1
        n = (d-1.0)/2.0
        return a + m - (((a+n)*x)/lower_incomplete_gamma(a,x,d=d+1))
    else:
        m = d-1
        n = d/2.0
        return a+m+((n*x)/(lower_incomplete_gamma(a,x,d=d+1)))


In [ ]:
def gammainc (a,b):
    from math import gamma
    return lower_incomplete_gamma(a,b)/gamma(a)

In [6]:
def cdf2Z(p):
    c=p
    """ P probably betwen 0 and 1 """
    if p<=0 or p>1 :
        print("p={} is out of range".format(c))
        return float("inf")
    if (c-0.5)>0:
        c=1. - c
    if (c-0.5)<0:
        #print(c)
        dt2= -2*log(c)
        t=sqrt(dt2)
        ppf= t- ((.010328*t + .802853)*t+ 2.515517) /\
             (((.001308*t + .189269)*t + 1.432788)*t + 1.)
        if p<0.5:
            ppf=-ppf
        return ppf
    if c-0.5==0:
        return 0

In [7]:
from collections import Counter
def SPI_function(x, fit=gammafit):
    zeros=Counter(x)[0]
    pzero=zeros/len(x) 
    x2=list(x[:])
    while 0 in x2 :
        x2.remove(0)
    if fit== gammafit:
        
        k,theta=fit(x2)# apply fit function
        print(pzero)
        if pzero<1:
            cdf1=[cdf2Z(pzero + (1.-pzero) *gammainc(k,i/theta)) for i in x]
        else:
            cdf1=[cdf2Z(pzero-0.0001 ) for i in x]
            
    print(str(fit.__name__),
          "shape ={}, scale= {}, pzero={}".format(k,theta,pzero))
    return cdf1

In [8]:
import io, csv
from collections import defaultdict
def readcsv(infile):
    """input file name, csv format"""
    with io.open(infile, newline='') as csvfile:
            data = defaultdict(list)
            reader = csv.DictReader(csvfile)
            for row in reader:
                for key in  row:
                    try:
                        data[key].append(dateparser(row[key]))
                    except:
                        if row[key]=="NA":
                            row[key]=float("nan")
                        data[key].append(float(row[key]))  
    return data

In [9]:
def write (data, outfile, fields=None):
    with io.open(outfile, 'w', newline='') as csvfile:
        if fields==None:
            fields=list(data.keys())
            print(fields)
        csvWriter = csv.DictWriter(csvfile, fieldnames=fields)
        csvWriter.writeheader()
    with io.open(outfile, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='"',
                            quoting=csv.QUOTE_MINIMAL)
        for  row in list(zip(*data.values())):
            row=list(row)
            row[0]=row[0].strftime("%Y-%m-%d")
            #print(row)
            writer.writerow(row)
##            if not row:
##                print("empty")
    print("The result is saved as {}".format(outfile))

In [10]:
from itertools import groupby
def group(indata,indate,func=None, fit=None):
    zdata=list(zip(indata,indate))
    grp=groupby(zdata, lambda x: x[1].month)
    P=defaultdict(list)
    gdates= defaultdict(list)
    for i, g in grp:
            for v , d in g:
                    gdates[i].append(d)
                    P[i].append(v)
    res=defaultdict(list)
    for key  in P:
            spiout=func(P[key],fit=fit)
            kdates=gdates[key]
            for d in kdates:
                    res["date"].append(d)
            for v in spiout:
                res["spi"].append(v)
    zdata=list(zip(res["spi"],res["date"]))
    zdata.sort(key=lambda x :x[1])
    res.clear()
    for  k in zdata:
            res["spi"].append(k[0])
            res["date"].append(k[1])
    sp,dat=res["spi"],res["date"]
    return sp,dat

In [12]:

from collections import  defaultdict
class SPI (object):
    def __init__(self, indata,dates,scales):
        self.scales=scales
        self.rain=indata
        self.dates=dates
    def calculate(self,fit=None):
        spiData=defaultdict(list)
        spiData["date"]=self.dates
        for scale in self.scales:
            dates2=self.dates[scale-1:]
            raindata=summov(self.rain,scale)
            spis,groupdate=group(raindata,dates2,func=util.SPI_function, fit=fit)
            lags= [float("nan") for i in range(scale-1)]
            spiData[scale]=lags+spis
        self.fields=spiData.keys()
        return spiData

In [16]:
#run1.py
##import sys
##sys.path.append(r"D:\spi")
import os
os.chdir(r"D:\spi/pydrought")
from pydrought import SPI
from pydrought.util import readcsv,write
import util
data=r"data\Rain.csv"
data=readcsv(data)
rain,dates=data["RAIN"],data["date"]
spi=SPI(rain,dates,[1,3,6])
spivalue=spi.calculate(fit=util.gammafit)
write(spivalue,r"D:\spi\pydrought\spi.csv")

0.0
gammafit shape =3.434454830234581, scale= 9.851151057654887, pzero=0.0
0.0
gammafit shape =4.600680924460504, scale= 8.813912693750886, pzero=0.0
0.0
gammafit shape =4.454341342077246, scale= 11.080326307579066, pzero=0.0
0.0
gammafit shape =2.0083821811361875, scale= 21.71731410303179, pzero=0.0
0.0
gammafit shape =1.0215820760193608, scale= 19.082602271572146, pzero=0.0
0.0
gammafit shape =0.7961488396912502, scale= 25.204667351457303, pzero=0.0
0.16666666666666666
gammafit shape =0.7573845798561917, scale= 26.705939366374743, pzero=0.16666666666666666
0.16666666666666666
gammafit shape =0.7778806857489468, scale= 16.429254812639247, pzero=0.16666666666666666
0.05555555555555555
gammafit shape =0.611202280895485, scale= 39.92131698895341, pzero=0.05555555555555555
0.0
gammafit shape =1.7986888280665014, scale= 16.36377169528131, pzero=0.0
0.0
gammafit shape =2.6921303193273163, scale= 13.851071258023751, pzero=0.0
0.0
gammafit shape =2.163314489246653, scale= 14.525036640957648, 

In [17]:
#run2.py
import os
inf=r"D:\spi\pydrought\data\Rain.csv"
cols="RAIN date"
outf=r"D:\spi\pydrought\data\spiss2.csv"
scales="1 3 6 9 "
cmd=r"D:\spi\pydrought\spi.py"
py="python"
expression=" ".join([py,cmd, inf, cols,outf ,scales])         
os.system(expression)

0

In [18]:
import os
inf=r"D:\spi\pydrought\data\Rain.csv"
cols="RAIN date"
outf=r"D:\spi\pydrought\data\spiss2.csv"
scales="1 3 6 9 "
cmd=r"D:\spi\pydrought\spi.py"
py="python"
expression=" ".join([py,cmd, inf, cols,outf ,scales])         
#os.system(expression)

import subprocess 
try:
    with open('output.txt', 'wb') as f:
        subprocess.check_call(expression, stdout=f, stderr=f)
except subprocess.CalledProcessError as error:
    print (error)
    with open('output.txt') as f:
        for line in f:
            print (line,)
##        